<a href="https://colab.research.google.com/github/pradeep2c1/Machine-Translation-model/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

import re

In [2]:
!nvidia-smi

Tue Aug 27 11:48:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.25                 Driver Version: 536.25       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   44C    P8               2W /  20W |    824MiB /  4096MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
tf.test.gpu_device_name()

''

# Data pre-processing

In [4]:
lines = pd.read_csv("Hindi_English_Truncated_Corpus.csv", encoding='utf-8')

In [5]:
lines['source'].value_counts()

source
tides        50000
ted          39881
indic2012    37726
Name: count, dtype: int64

In [6]:
lines = lines[lines['source'] == 'ted']
lines.head(10)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
23,ted,This changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
30,ted,"And you can see, this LED is going to glow.","और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।"
32,ted,to turn on the lights or to bring him a glass ...,"लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,"
35,ted,Can you imagine saying that?,क्या आप ये कल्पना कर सकते है


In [7]:
lines.drop(columns = ['source'], inplace = True)
lines.shape

(39881, 2)

In [8]:
lines.reset_index(inplace = True)
lines.drop(columns = ['index'], inplace = True)

In [9]:
pd.isnull(lines).sum()

english_sentence    0
hindi_sentence      0
dtype: int64

In [11]:
lines.drop_duplicates(inplace=True)
lines.shape

(38803, 2)

In [12]:
lines.head()

,english_sentence,hindi_sentence
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
3,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
4,So there is some sort of justice,तो वहाँ न्याय है


In [15]:
# Make all english letters lowercase
lines['english_sentence'] = lines['english_sentence'].apply(lambda x: x.lower())

# Remove the quotes
lines['english_sentence'] = lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

# Remove special characters
sp_char = set(string.punctuation)
lines['english_sentence'] = lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in sp_char))
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in sp_char))

In [16]:
# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [17]:
# Add START and END tokens to the beginning and end of the target sequence
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: 'START_ ' + x + ' _END')

In [18]:
lines.head()

,english_sentence,hindi_sentence
0,politicians do not have permission to do what ...,START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...
1,id like to tell you about one such child,START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...
2,what we really mean is that theyre bad at not ...,START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...
3,and who are we to say even that they are wrong,START_ और हम होते कौन हैं यह कहने भी वाले कि व...
4,so there is some sort of justice,START_ तो वहाँ न्याय है _END


In [19]:
lines = lines.sample(frac = 1).reset_index(drop = True)
lines.head()

,english_sentence,hindi_sentence
0,we developed these novel actuators for joints,START_ हमने जोड़ो के लिए उत्तम actuators बनाये...
1,and we would see ourselves on tv pasting,START_ और हम अपने आपको टीवी पर देखते तसवीरें च...
2,and heres an example of a file,START_ और यहाँ एक फ़ाइल का एक उदाहरण है _END
3,and calculate the control commands,START_ और नियंत्रण आदेशो की गणना करता है _END
4,this which is helicodiceros,START_ यह हेलिकोदिक्रोस है _END


# Tokenizing the data

In [20]:
en = set()
for sentence in lines['english_sentence']:
    for word in sentence.split():
        if word not in en:
            en.add(word)

hi = set()
for sentence in lines['hindi_sentence']:
    for word in sentence.split():
        if word not in hi:
            hi.add(word)

In [21]:
print('Unique words in English are : ', len(en))
print('Unique words in Hindi are : ', len(hi))

Unique words in English are :  17759
Unique words in Hindi are :  22782


In [22]:
lines['length_en'] = lines['english_sentence'].apply(lambda x: len(x.split()))
lines['length_hi'] = lines['hindi_sentence'].apply(lambda x: len(x.split()))

lines.head()

,english_sentence,hindi_sentence,length_en,length_hi
0,we developed these novel actuators for joints,START_ हमने जोड़ो के लिए उत्तम actuators बनाये...,7,10
1,and we would see ourselves on tv pasting,START_ और हम अपने आपको टीवी पर देखते तसवीरें च...,8,12
2,and heres an example of a file,START_ और यहाँ एक फ़ाइल का एक उदाहरण है _END,7,10
3,and calculate the control commands,START_ और नियंत्रण आदेशो की गणना करता है _END,5,9
4,this which is helicodiceros,START_ यह हेलिकोदिक्रोस है _END,4,5


In [23]:
print(lines[lines['length_en'] > 20].shape)
print(lines[lines['length_hi'] > 20].shape)

(1, 4)
(334, 4)


In [24]:
lines = lines[lines['length_en'] <= 20]
lines = lines[lines['length_hi'] <= 20]
print(lines.shape)

(38469, 4)


In [25]:
print("Maximum sequence length of inputs =", max(lines['length_en']))
print("Maximum sequence length of outputs =", max(lines['length_hi']))

Maximum sequence length of inputs = 20
Maximum sequence length of outputs = 20


In [26]:
max_encoder_seq_length = max(lines['length_en'])
max_decoder_seq_length = max(lines['length_hi'])

input_words = sorted(list(en))
target_words = sorted(list(hi))
num_encoder_tokens = len(en)
num_decoder_tokens = len(hi)
num_encoder_tokens, num_decoder_tokens

(17759, 22782)

In [28]:
num_encoder_tokens += 1
num_decoder_tokens += 1

In [29]:
input_token_id = dict([(word, i + 1) for i, word in enumerate(input_words)])
target_token_id = dict([(word, i + 1) for i, word in enumerate(target_words)])

rev_input_char_id = dict((i, word) for word, i in input_token_id.items())
rev_target_char_id = dict((i, word) for word, i in target_token_id.items())

In [30]:
lines = shuffle(lines)
lines.head()

,english_sentence,hindi_sentence,length_en,length_hi
27537,and this is where we are today,START_ और हम यहाँ है अभी। _END,7,7
11657,where passengers on the ferry,START_ जहाँ फेरी के यात्रियों ने _END,5,7
5812,where no sentient creature is harmed,START_ जहां कोई संवेदनशील प्राणी को नुकसान नही...,6,11
4695,doesnt feel like anything,START_ कुछ भी महसूस नहीं करता है _END,4,8
9597,it would be necessary to look at the brain dir...,START_ मस्तिष्क को सीधे ही देखना आवश्यक होना च...,10,10


# Splitting the data set

In [32]:
# Splitting data set into 80/20 for train/test data
split = int(0.8 * len(lines))

train_set = lines[: split]
test_set = lines[split :]

x_train, y_train = train_set['english_sentence'], train_set['hindi_sentence']
x_test, y_test = test_set['english_sentence'], test_set['hindi_sentence']

x_train.shape, x_test.shape

((30775,), (7694,))

In [34]:
def generate_batch(x = x_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(x), batch_size):
            encoder_input_data = np.zeros((batch_size, max_encoder_seq_length), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_decoder_seq_length), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_decoder_seq_length, num_decoder_tokens), dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(x[j:j + batch_size], y[j:j + batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_id[word]   # encoder input seq
                for t, word in enumerate(target_text.split()):    
                    if t < len(target_text.split()) - 1:
                        decoder_input_data[i, t] = target_token_id[word]   # decoder input seq
                    if t > 0:
                        # does not include the START_ token
                        decoder_target_data[i, t - 1, target_token_id[word]] = 1. 
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Making the Model

In [35]:
latent_dim = 256

In [36]:
encoder_inputs = Input(shape=(None,))  # Create the input tensor
encoder_embedding =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
# We discard encoder_outputs and only keep the states
encoder_states = [state_h, state_c]

In [37]:
# Set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
 
# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [39]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            4546816   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            5832704   ['input_2[0][0]']             
                                                                                              

In [40]:
train_samples = len(x_train)
val_samples = len(x_test)
batch_size = 128
epochs = 100

In [41]:
model.fit(
    generate_batch(x_train, y_train, batch_size = batch_size),
    steps_per_epoch = train_samples//batch_size,
    epochs = epochs,
    validation_data = generate_batch(x_test, y_test, batch_size = batch_size),
    validation_steps = val_samples//batch_size)

Epoch 1/100
240/240 [==============================] - 419s 2s/step - loss: 6.6972 - val_loss: 6.2043
Epoch 2/100
240/240 [==============================] - 384s 2s/step - loss: 5.9052 - val_loss: 5.8430
Epoch 3/100
240/240 [==============================] - 378s 2s/step - loss: 5.5250 - val_loss: 5.6480
Epoch 4/100
240/240 [==============================] - 373s 2s/step - loss: 5.2632 - val_loss: 5.5371
Epoch 5/100
240/240 [==============================] - 374s 2s/step - loss: 5.0377 - val_loss: 5.4517
Epoch 6/100
240/240 [==============================] - 377s 2s/step - loss: 4.8278 - val_loss: 5.3868
Epoch 7/100
240/240 [==============================] - 380s 2s/step - loss: 4.6203 - val_loss: 5.3216
Epoch 8/100
240/240 [==============================] - 376s 2s/step - loss: 4.4187 - val_loss: 5.2864
Epoch 9/100
240/240 [==============================] - 377s 2s/step - loss: 4.2216 - val_loss: 5.2666
Epoch 10/100
240/240 [==============================] - 380s 2s/step - loss: 4.031

In [42]:
model.save_weights('nmt_weights.h5')

In [43]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate probability over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [44]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_id['START_']

    # Sampling loop for a batch of sequences
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = rev_target_char_id[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or len(decoded_sentence) > 50):
            break

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [45]:
train_gen = generate_batch(x_train, y_train, batch_size = 1)
k=-1

In [46]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 36ms/step
Input English sentence: and this is where we are today
Actual Hindi Translation:  और हम यहाँ है अभी। 
Predicted Hindi Translation:  और हम यहाँ है अभी। 


In [47]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 30ms/step
Input English sentence: where passengers on the ferry
Actual Hindi Translation:  जहाँ फेरी के यात्रियों ने 
Predicted Hindi Translation:  जहाँ फेरी के यात्रियों ने 


In [48]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 26ms/step
Input English sentence: where no sentient creature is harmed
Actual Hindi Translation:  जहां कोई संवेदनशील प्राणी को नुकसान नहीं पहुंचाया जाते 
Predicted Hindi Translation:  जहां कोई संवेदनशील प्राणी को नुकसान नहीं पहुंचाया 


In [49]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 27ms/step
Input English sentence: doesnt feel like anything
Actual Hindi Translation:  कुछ भी महसूस नहीं करता है 
Predicted Hindi Translation:  कुछ भी महसूस नहीं करते 


In [51]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 31ms/step
Input English sentence: in improving the technology of the heart
Actual Hindi Translation:  अपने हृदय की तकनीक के संवर्धन में लगाएं 
Predicted Hindi Translation:  अपने हृदय की तकनीक के गहरे दिए जाते हैं 
